In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import requests 
import pandas as pd
import os
from google.colab import drive
import numpy 

drive.mount('/content/drive')

# Función para obtener datos desde la API
def fetch_insights(url, params):
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:

            print(response.json())
            return response.json()
        elif response.status_code == 400:
            error_data = response.json().get("error", {})
            if error_data.get("code") == 100 and "last 30 days excluding the current day" in error_data.get("message", ""):
                print("La métrica 'follower_count' solo está disponible para los últimos 30 días. Revisa el rango de fechas.")
            else:
                raise Exception(f"Error desconocido: {response.status_code}, {response.text}")
        else:
            raise Exception(f"Error al consultar la API: {response.status_code}, {response.text}")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error en la conexión: {e}")

# Función para procesar los datos de la API y convertirlos al formato solicitado
def process_data(data):
    insights = data["data"][0]["values"]
    df = pd.DataFrame(insights)
    df["end_time"] = pd.to_datetime(df["end_time"])

    # Transformar al formato solicitado
    df["Date"] = df["end_time"].dt.strftime('%d/%m/%Y')
    df["Month"] = df["end_time"].dt.strftime('%B')
    df["Day of the Week"] = df["end_time"].dt.strftime('%A')
    df.rename(columns={"value": "Daily Followers"}, inplace=True)
    df["Accumulated Followers"] = None  # Inicializar vacío

    # Reordenar columnas
    df = df[["Month", "Date", "Day of the Week", "Daily Followers", "Accumulated Followers"]]
    return df

# Función para actualizar o crear el CSV
def update_csv(new_data, csv_file):
    os.makedirs(os.path.dirname(csv_file), exist_ok=True)

    # Cargar el archivo existente si está disponible
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
        existing_data["Date"] = pd.to_datetime(existing_data["Date"], format='%Y-%m-%d')
    else:
        existing_data = pd.DataFrame(columns=["Month", "Date", "Day of the Week", "Daily Followers", "Accumulated Followers"])

    # Convertir las fechas en el nuevo conjunto de datos al mismo formato
    new_data["Date"] = pd.to_datetime(new_data["Date"], format='%d/%m/%Y')

    # Reemplazar valores de `Daily Followers` con 0 por nuevos datos si son diferentes de 0
    for index, row in new_data.iterrows():
        matching_row = existing_data[existing_data["Date"] == row["Date"]]
        if not matching_row.empty:
            if matching_row.iloc[0]["Daily Followers"] == 0 and row["Daily Followers"] != 0:
                existing_data.loc[matching_row.index, "Daily Followers"] = row["Daily Followers"]
        else:
            # Si no hay fila existente para esta fecha, añadirla
            existing_data = pd.concat([existing_data, pd.DataFrame([row])], ignore_index=True)

      # Ordenar por fecha y guardar el resultado en el archivo CSV
    combined_data = existing_data.sort_values("Date").drop_duplicates(subset="Date")
    combined_data["Accumulated Followers"] = combined_data["Daily Followers"].cumsum() + 5764
    combined_data.to_csv(csv_file, index=False)
    return combined_data


# Función principal para manejar la paginación y guardar datos
def main():
    url = BASE_URL
    all_data = pd.DataFrame(columns=["Month", "Date", "Day of the Week", "Daily Followers", "Accumulated Followers"])

    while url:
        data = fetch_insights(url, PARAMS)
        if not data:
            break
        new_data = process_data(data)
        all_data = pd.concat([all_data, new_data], ignore_index=True)

        # Obtener la URL de la siguiente página si existe
        url = data.get("paging", {}).get("previous", None)

    # Actualizar o crear el archivo CSV
    updated_data = update_csv(all_data, CSV_FILE)
    print("Datos actualizados exitosamente.")
    print(updated_data)

# Ejecutar el script
if __name__ == "__main__":
    # URL base de la API
    BASE_URL = "https://graph.facebook.com/v21.0/17841409297279102/insights"
    ACCESS_TOKEN = "EABCSxik8s3MBO8eRV4tn35D1F04QlQjsHVpAgI8fm6xPgDtHjOu3moesB15eiRuH8EzwToIGIdMLf3P14gVDxpuVFlv2juBxTCA1jSF0xOGofBNypKvTDu2X5qJNu7ExpDZAjX1Dbvz3HCH0RIUSZCqXMWpnPbOMILRxrpDZBS7eKLmAGd2vaTT3R4jLtqEsqJDQ6K4JRqQWVgFWaoLKVURzwZDZD"
    PARAMS =  {
        "metric": "impressions,reach,total_interactions,accounts_engaged,likes,comments,saves,shares,replies,follows_and_unfollows,profile_links_taps",
        "period": "day",
        "metric_type": "total_value",
        "access_token": ACCESS_TOKEN,
    }

    # Nombre del archivo CSV
    CSV_FILE = "/content/drive/MyDrive/LORD_CONEJO/merged_lordconejo_data.csv"
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'data': [{'name': 'impressions', 'period': 'day', 'title': 'Impressions', 'description': 'The number of times that your posts, stories, reels, videos and live videos were on screen, including in ads.', 'total_value': {'value': 4024}, 'id': '17841409297279102/insights/impressions/day'}, {'name': 'reach', 'period': 'day', 'title': 'Accounts reached', 'description': 'The number of unique accounts that have seen your content, at least once, including in ads. Content includes posts, stories, reels, videos and live videos. Reach is different from impressions, which may include multiple views of your content by the same accounts. This metric is estimated and in development.', 'total_value': {'value': 2859}, 'id': '17841409297279102/insights/reach/day'}, {'name': 'total_interactions', 'period': 'day', 'title': 'Content interactions', 'description': 'The total number

KeyError: 'values'